In [18]:
import pandas as pd

uri = "https://gist.githubusercontent.com/guilhermesilveira/e99a526b2e7ccc6c3b70f53db43a87d2/raw/1605fc74aa778066bf2e6695e24d53cf65f2f447/machine-learning-carros-simulacao.csv"
dados = pd.read_csv(uri).drop(columns=["Unnamed: 0"], axis=1)
dados.head()

,preco,vendido,idade_do_modelo,km_por_ano
0,30941.02,1,18,35085.22134
1,40557.96,1,20,12622.05362
2,89627.50,0,12,11440.79806
3,95276.14,0,3,43167.32682
4,117384.68,1,4,12770.11290


In [19]:
import  numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score

x = dados[["preco", "idade_do_modelo", "km_por_ano"]]
y = dados["vendido"]

seed = 158020
np.random.seed(seed)
treino_x, teste_x, treino_y, teste_y = train_test_split(x, y, test_size=0.25, stratify=y)

print("Treinamos com %d elementos e testamos com %d elementos" % (len(treino_x), len(teste_x)))

Treinamos com 7500 elementos e testamos com 2500 elementos


In [20]:
from sklearn.dummy import DummyClassifier

dummy_stratifeid = DummyClassifier()
dummy_stratifeid.fit(treino_x, treino_y)
acuracia = dummy_stratifeid.score(teste_x, teste_y) * 100

print("Acurácia: %.2f" % acuracia)

Acurácia: 58.00


In [21]:
from sklearn.tree import DecisionTreeClassifier

seed = 158020
np.random.seed(seed)
modelo = DecisionTreeClassifier(max_depth=2)
modelo.fit(treino_x, treino_y)
previsoes = modelo.predict(teste_x)

acuracia = accuracy_score(teste_y, previsoes) * 100
print("Acurácia: %.2f" % acuracia)

Acurácia: 71.92


In [22]:
from sklearn.model_selection import cross_validate

seed = 158020
np.random.seed(seed)
modelo = DecisionTreeClassifier(max_depth=2)
resultados = cross_validate(modelo, x, y, cv=5, return_train_score=False)
media = resultados["test_score"].mean()
desvio_padrao = resultados["test_score"].std()
print("Accuracy com cross validation, 5 = [%.2f, %.2f]" % ((media - 2 *desvio_padrao) * 100, (media + 2 * desvio_padrao) * 100))

Accuracy com cross validation, 5 = [75.21, 76.35]


# Aloriatoriadade no cross validation

In [23]:
def print_results(results):
    media = resultados["test_score"].mean()
    desvio_padrao = resultados["test_score"].std()
    print("Accuracy com cross validation = [%.2f, %.2f]" % ((media - 2 *desvio_padrao) * 100, (media + 2 * desvio_padrao) * 100))

In [24]:
from sklearn.model_selection import cross_validate
from sklearn.model_selection import KFold

seed = 158020
np.random.seed(seed)
cv = KFold(n_splits=10, shuffle=True, random_state=seed)

modelo = DecisionTreeClassifier(max_depth=2)
resultados = cross_validate(modelo, x, y, cv=cv, return_train_score=False)
print_results(resultados)

Accuracy com cross validation = [73.58, 77.98]


# Estratificaçao
### Simular azar
Pode ser uma situação de "azar" ou uma proporção de exemplos desbalanceado entre as classes.

In [25]:
dados_com_azar = dados.sort_values("vendido", ascending=True)
x_azar = dados_com_azar[["preco", "idade_do_modelo", "km_por_ano"]]
y_azar = dados_com_azar["vendido"]
dados_com_azar.head()

,preco,vendido,idade_do_modelo,km_por_ano
4999,74023.29,0,12,24812.80412
5322,84843.49,0,13,23095.63834
5319,83100.27,0,19,36240.72746
5316,87932.13,0,16,32249.56426
5315,77937.01,0,15,28414.50704


In [26]:
from sklearn.model_selection import cross_validate
from sklearn.model_selection import KFold

seed = 158020
np.random.seed(seed)
cv = KFold(n_splits=10)

modelo = DecisionTreeClassifier(max_depth=2)
resultados = cross_validate(modelo, x_azar, y_azar, cv=cv, return_train_score=False)
print_results(resultados)

Accuracy com cross validation = [34.29, 81.39]


In [27]:
from sklearn.model_selection import cross_validate
from sklearn.model_selection import KFold

seed = 158020
np.random.seed(seed)
cv = KFold(n_splits=10, shuffle=True)

modelo = DecisionTreeClassifier(max_depth=2)
resultados = cross_validate(modelo, x_azar, y_azar, cv=cv, return_train_score=False)
print_results(resultados)

Accuracy com cross validation = [73.02, 78.50]


In [28]:
from sklearn.model_selection import cross_validate
from sklearn.model_selection import StratifiedKFold

seed = 158020
np.random.seed(seed)
cv = StratifiedKFold(n_splits=10, shuffle=True)

modelo = DecisionTreeClassifier(max_depth=2)
resultados = cross_validate(modelo, x_azar, y_azar, cv=cv, return_train_score=False)
print_results(resultados)

Accuracy com cross validation = [74.10, 77.46]


# Dados agrupaveis

In [29]:
dados.idade_do_modelo

0       18
1       20
2       12
3        3
4        4
        ..
9995    12
9996    16
9997     4
9998     7
9999    19
Name: idade_do_modelo, Length: 10000, dtype: int64

In [30]:
np.random.seed(seed)
rand = np.random.randint(-2, 3, size=10000)
dados['modelos_aleatorios'] = dados.idade_do_modelo + rand
dados

,preco,vendido,idade_do_modelo,km_por_ano,modelos_aleatorios
0,30941.02,1,18,35085.22134,17
1,40557.96,1,20,12622.05362,20
2,89627.50,0,12,11440.79806,12
3,95276.14,0,3,43167.32682,2
4,117384.68,1,4,12770.11290,2
...,...,...,...,...,...
9995,97112.86,0,12,25060.64248,13
9996,107424.63,1,16,21317.31764,18
9997,93856.99,0,4,20950.38812,3
9998,51250.57,1,7,16840.13376,9


# Dados agrupaveis
### continuaçao

In [39]:
from sklearn.model_selection import GroupKFold

seed = 301
np.random.seed(seed)

cv = GroupKFold(n_splits=10)
modelo = DecisionTreeClassifier(max_depth=2)
resultados = cross_validate(modelo, x_azar, y_azar, cv=cv, return_train_score=False, groups= dados.modelos_aleatorios)
print_results(resultados)

Accuracy com cross validation = [72.91, 78.60]


# Pipeline

### Crossvalidation com StanderScaler

In [41]:
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

scaler = StandardScaler()
scaler.fit(treino_x)
treino_x_escalado = scaler.transform(treino_x)
teste_x_esccalado = scaler.transform(teste_x)

modelo = SVC()
modelo.fit(treino_x_escalado, treino_y)
previsoes = modelo.predict(teste_x_esccalado)

acuracia = accuracy_score(teste_y, previsoes) * 100
print(acuracia)

74.4


In [ ]:
from  sklearn.pipeline import Pipeline

scaler = StandardScaler()
modelo = SVC()

pipeline = Pipeline([('transformacao', scaler), ('estimador', modelo)])

cv = GroupKFold(n_splits = 10)
results = cross_validate(pipeline, x_azar, y_azar, cv = cv, groups = dados.modelos_aleatorios, return_train_score=False)
print_results(results)